In [ ]:
# LeetCode 208: Implement Trie (Prefix Tree)
# https://leetcode.com/problems/implement-trie-prefix-tree/
# Time Complexity: O(L)
# Space Complexity: Worst case	O(n * L), Best case	O(L)
# n = number of words inserted
# L = average length of the words

# 208. Implement Trie (Prefix Tree)

[Link to Problem](https://leetcode.com/problems/implement-trie-prefix-tree/description/)

### Description
A **trie** (pronounced as "try") or **prefix tree** is a tree data structure used to efficiently store and retrieve keys in a dataset of strings. There are various applications of this data structure, such as autocomplete and spellchecker.

Implement the `Trie` class:
- `Trie()` Initializes the trie object.
- `void insert(String word)` Inserts the string `word` into the trie.
- `boolean search(String word)` Returns `true` if the string `word` is in the trie (i.e., was inserted before), and `false` otherwise.
- `boolean startsWith(String prefix)` Returns `true` if there is a previously inserted string `word` that has the prefix `prefix`, and `false` otherwise.

---
**Example:**

Input:
`["Trie", "insert", "search", "search", "startsWith", "insert", "search"]`
`[[], ["apple"], ["apple"], ["app"], ["app"], ["app"], ["app"]]`

Output:
`[null, null, true, false, true, null, true]`

Explanation:
`Trie trie = new Trie();`
`trie.insert("apple");`
`trie.search("apple"); // return True`
`trie.search("app"); // return False`
`trie.startsWith("app"); // return True`
`trie.insert("app");`
`trie.search("app"); // return True`

---
**Constraints:**
- `1 <= word.length, prefix.length <= 2000`
- `word` and `prefix` consist only of lowercase English letters.
- At most `3 * 10^4` calls in total will be made to `insert`, `search`, and `startsWith`.

In [21]:
class Trie:
    def __init__(self, root=None):
        self.root = root
        self.children = []

    def insert(self, word: str) -> None:
        if word == "":
            if not self.search(word):
                self.children.append(Trie())
            return None
        
        char = word[0]
        for node in self.children:
            if node.root == char:
                return node.insert(word[1:])

        new_node = Trie(char)
        self.children.append(new_node)
        return new_node.insert(word[1:])
        # Time: O(L²) worst case (due to list scan)

    def search(self, word: str) -> bool:
        if word == "":
            for node in self.children:
                if node.root == None:
                    return True
            return False
            
        char = word[0]
        for node in self.children:
            if node.root == char:
                return node.search(word[1:])
        return False
        # Time: O(L²)

    def startsWith(self, prefix: str) -> bool:
        if prefix == "":
            return True
            
        char = prefix[0]
        for node in self.children:
            if node.root == char:
                return node.startsWith(prefix[1:])
        return False
        # Time: O(L²)

# L = length of the input string	

## ✅ **Strengths**

| Aspect                 | Comments                                                                       |
| ---------------------- | ------------------------------------------------------------------------------ |
| **Correctness**        | The implementation passes all standard use cases (insert, search, startsWith). |
| **Recursive style**    | Clean recursion makes logic easier to follow conceptually.                     |
| **Edge case handling** | You handle empty strings, which many implementations ignore.                   |
| **Test coverage**      | Basic tests for key operations are included.                                   |

---

## 🛠️ **Areas for Improvement**

### 1. 🔄 Inefficient Data Structure (`self.children` as list)

| Problem | Using a `list` for `children` means every character lookup is **O(n)** time, where `n` is the number of children. |
| ------- | ----------------------------------------------------------------------------------------------------------------- |
| Fix     | Replace `self.children` with a `dict`, mapping characters to `Trie` nodes for **O(1)** lookup.                    |

```python
self.children = {}  # instead of []
```

---

### 2. 🚫 Unusual Use of `root` to Track Characters

| Problem | `self.root` stores the character of the node, but it’s redundant if the `children` dict is used.                          |
| ------- | ------------------------------------------------------------------------------------------------------------------------- |
| Fix     | Eliminate `root` entirely. You don’t need to store the character in the node—it's already in the parent’s dictionary key. |

---

### 3. 🟡 Verbose Recursive Pattern

\| Observation | The recursion is clear, but it becomes verbose with repeated slicing (`word[1:]`) and manual character checks. |
\| Suggestion | While recursion is acceptable, an iterative implementation is usually more space-efficient and readable.

---

### 4. ⚠️ Ambiguous Handling of Empty Strings

\| Issue | The use of `Trie(None)` to represent word termination is unconventional. It may be unclear to future readers. |
\| Fix | Use an explicit flag like `is_end_of_word` instead.

```python
class Trie:
    def __init__(self):
        self.children = {}
        self.is_end = False
```

---

### ✅ **Recommended Refactor**

Here’s a **clean, efficient version** using `dict` and a termination flag:

In [22]:
class Trie:
    def __init__(self):
        self.children = {}
        self.is_end = False

    def insert(self, word: str) -> None:
        node = self
        for ch in word:
            if ch not in node.children:
                node.children[ch] = Trie()
            node = node.children[ch]
        node.is_end = True

    def search(self, word: str) -> bool:
        node = self
        for ch in word:
            if ch not in node.children:
                return False
            node = node.children[ch]
        return node.is_end

    def startsWith(self, prefix: str) -> bool:
        node = self
        for ch in prefix:
            if ch not in node.children:
                return False
            node = node.children[ch]
        return True

## ⏱️ **Performance**

| Operation                        | Your Version (List)                 | Recommended (Dict) |
| -------------------------------- | ----------------------------------- | ------------------ |
| `insert()`                       | O(L²) worst case (due to list scan) | O(L)               |
| `search()`                       | O(L²)                               | O(L)               |
| `startsWith()`                   | O(L²)                               | O(L)               |
| *L = length of the input string* |                                     |                    |

---

## 🧪 Test Enhancements

Add more edge cases:

```python
# Insert/search empty string
assert obj.insert("") == None
assert obj.search("") == True

# Prefix that matches a whole word
assert obj.startsWith("apple") == True
assert obj.startsWith("applf") == False
```

---

## ✅ Summary

| Category       | Rating / Comment                                          |
| -------------- | --------------------------------------------------------- |
| Correctness    | ✅ Passes tests                                            |
| Efficiency     | ⚠️ Needs dict-based optimization                          |
| Readability    | 🔄 Recursion is clear, but could be simplified            |
| Scalability    | ❌ Current list-based child handling won’t scale well      |
| Recommendation | ✅ Use a `dict` for children and `is_end` flag for clarity |

In [23]:
# Test
word1 = "apple"
word2 = "app"
obj = Trie()
assert obj.insert(word1) == None
assert obj.search(word1) == True
assert obj.search(word2) == False
assert obj.startsWith(word2) == True
assert obj.insert(word2) == None
assert obj.search(word2) == True